### this notebook scrapes precinct-delimited zip files for each county

In [1]:
import os
import time
import shutil
from selenium import webdriver

download_dir = '../../../Downloads'
target_dir = 'zips/'
max_down_wait_sec = 60

#### get all county result page URLs

In [2]:
url = 'https://results.enr.clarityelections.com/GA/105369/web.264614/#/access-to-races'
d = webdriver.Chrome()
d.get(url)
time.sleep(3)
d.execute_script("window.scrollTo(0, 1200)") 
time.sleep(1)
d.execute_script("window.scrollTo(0, 1500)") 
time.sleep(1)

links = d.find_elements_by_tag_name('a')
links = [l.get_attribute('href') for l in links]
d.quit()
county_urls = {h.split("//GA/")[1].split('/')[0]: h
               for h in links if "//GA/" in str(h)}
county_urls = {k:v for k, v in county_urls.items() if k!=''}

# visit county page & download data

In [3]:
def scrape_county(county_name):
    target_fname = county_name
    url = county_urls[county_name] #key

    d = webdriver.Chrome()
    d.get(url)
    for dl_try in range(3):
        try:
            time.sleep(2)
            d.execute_script(f"window.scrollTo(0, 1050)")
            dl_btn = d.find_element_by_css_selector("a[download='detailtxt.zip']")
            dl_btn.click()
            time.sleep(1)
            break
        except: pass
    
    filename = max([f for f in os.listdir(download_dir)], # while downloading
                    key=lambda x : os.path.getctime(os.path.join(download_dir,x)))
    d.quit()
    waits = 0
    while '.part' in filename or 'crdownload' in filename: # still downloading
        if waits > max_down_wait_sec*2:
            print("\n  [ERROR] - download timed out (max waits exceeded)!")
            raise TimeoutError()
        # re-define filename to check for 'part' or 'crdownload' (not done)
        filename = max([f for f in os.listdir(download_dir)],
                   key=lambda xa : os.path.getctime(os.path.join(download_dir,xa)))
        waits += 1
        time.sleep(.5)
        

    # rename & move downloaded file to target directory
    county_name += '.zip'
    os.rename(os.path.join(download_dir, filename),
              os.path.join(download_dir, county_name))
    time.sleep(.15)
    print(county_name)
    shutil.move(os.path.join(download_dir, county_name),
                os.path.join(target_dir, county_name))
    print(f"> '{county_name}' downloaded to {target_dir}")


In [4]:
max_attempts = 3
for county in list(county_urls.keys()):
    attempts = 0
    done = False
    o_len = len(os.listdir(target_dir))
    while attempts <= max_attempts:
        if done: break
        attempts += 1
        try:
            scrape_county(county)
            time.sleep(2)
            if len(os.listdir(target_dir)) != o_len:
                done = True
        except:
            print('  > failed, retrying...')
            time.sleep(2)
            try: d.quit()
            except: pass

Appling.zip
> 'Appling.zip' downloaded to zips/
Atkinson.zip
> 'Atkinson.zip' downloaded to zips/
Bacon.zip
> 'Bacon.zip' downloaded to zips/
Baker.zip
> 'Baker.zip' downloaded to zips/
Baldwin.zip
> 'Baldwin.zip' downloaded to zips/
Banks.zip
> 'Banks.zip' downloaded to zips/
Barrow.zip
> 'Barrow.zip' downloaded to zips/
Bartow.zip
> 'Bartow.zip' downloaded to zips/
Ben_Hill.zip
> 'Ben_Hill.zip' downloaded to zips/
Berrien.zip
> 'Berrien.zip' downloaded to zips/
Bibb.zip
> 'Bibb.zip' downloaded to zips/
Bleckley.zip
> 'Bleckley.zip' downloaded to zips/
Brantley.zip
> 'Brantley.zip' downloaded to zips/
Brooks.zip
> 'Brooks.zip' downloaded to zips/
Bryan.zip
> 'Bryan.zip' downloaded to zips/
Bulloch.zip
> 'Bulloch.zip' downloaded to zips/
Burke.zip
> 'Burke.zip' downloaded to zips/
Butts.zip
> 'Butts.zip' downloaded to zips/
Calhoun.zip
> 'Calhoun.zip' downloaded to zips/
Camden.zip
> 'Camden.zip' downloaded to zips/
Candler.zip
> 'Candler.zip' downloaded to zips/
Carroll.zip
> 'Carroll

In [5]:
os.listdir('raw')

['.DS_Store']

In [6]:
#manual extra scrapes

In [7]:
scrape_county('Wilkinson')

Wilkinson.zip
> 'Wilkinson.zip' downloaded to zips/
